In [4]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (13.9s)

No dependencies to install or update


# Option valuation model based on historical prices

In [1]:
import importlib.util
import os
google_colab_spec = importlib.util.find_spec("google") and importlib.util.find_spec("google.colab")
if google_colab_spec:
    google_colab = importlib.import_module('google.colab')
    google_colab.drive.mount('/content/drive', force_remount=True)
    path = '/content/drive/MyDrive/Colab Notebooks/Ethereum Prices'
else:
    path = f'{os.environ["HOME"]}/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices'

import optionspricing
import binanceprices

COUNT_YEARS = 10
BINANCE_DATETIME_FORMAT = "%Y-%m-%d %H-%M-%S"
BINANCE_SYMBOL = 'ETHUSDT'

prices_df = binanceprices.load_prices(path, BINANCE_SYMBOL, COUNT_YEARS)



loading 2014 .no previous data found in /home/python/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices/ETHUSDT/2014, loading from binance
.no previous data found in /home/python/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices/ETHUSDT/2014, loading from binance
.no previous data found in /home/python/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices/ETHUSDT/2014, loading from binance
.no previous data found in /home/python/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices/ETHUSDT/2014, loading from binance
.no previous data found in /home/python/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices/ETHUSDT/2014, loading from binance
.no previous data found in /home/python/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Pr

## Option valuation model: input parameters here

In [6]:
target_period_hours = 3 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2022, 1)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

simulation = trading_model.evaluate(prices_df, strikes_universe_size)
print(simulation)
simulation.allocate([1, 2], 1500. * 2./100.)

target expiry: 2024-02-02 08:00:00+00:00 (72 hours left)
current price: 2310.99
-------------------------------
trading put 2250 and call 2350 (hit ratio: 45%)
cost: 0.014 / value: 0.027, benefit/cost = 1.9x
($) cost: 33.51 / value: 63.41, average gain = 29.90
-------------------------------
trading put 2200 and call 2400 (hit ratio: 32%)
cost: 0.007 / value: 0.017, benefit/cost = 2.5x
($) cost: 15.95 / value: 39.13, average gain = 23.18
-------------------------------
trading put 2150 and call 2450 (hit ratio: 21%)
cost: 0.004 / value: 0.010, benefit/cost = 3.0x
($) cost: 8.09 / value: 24.26, average gain = 16.17
-------------------------------
trading put 2100 and call 2500 (hit ratio: 14%)
cost: 0.002 / value: 0.007, benefit/cost = 3.0x
($) cost: 5.08 / value: 15.10, average gain = 10.01


[{'put': 2200.0, 'call': 2400.0, 'quantity': 1},
 {'put': 2150.0, 'call': 2450.0, 'quantity': 2}]

In [7]:
simulation.options_chain

,value_call,value_call_pct,call_ask,value_put,value_put_pct,put_ask
strike,,,,,,
2100.0,218.636304,NaN,1.0000,7.641673,0.003307,0.0008
2150.0,172.949599,NaN,1.0000,11.954968,0.005173,0.0013
2200.0,129.374657,NaN,0.0525,18.380027,0.007953,0.0026
2250.0,89.684222,NaN,0.0360,28.689592,0.012414,0.0055
2300.0,56.909161,NaN,0.0185,45.914531,0.019868,0.0120
2350.0,34.724747,0.015026,0.0090,73.730116,NaN,0.0250
2400.0,20.747206,0.008978,0.0043,109.752575,NaN,0.0450
2450.0,12.302602,0.005324,0.0022,151.307971,NaN,0.0650
2500.0,7.453503,0.003225,0.0014,196.458873,NaN,0.0855
